<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/Query_Decomposition/createSubQuery_by_conjunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai ragas datasets faiss-cpu groq langchain_community evaluate anls sentence_transformers
!pip install -U langchain-openai

In [ ]:
from google.colab import drive
from langchain.vectorstores import FAISS
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from google.colab import drive, userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('me_OPENAI_API_KEY')

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import json
import openai

# Load JSON data from file
#input_path = "/content/drive/MyDrive/Final/Dataset/MultiHop_RAG/multihop_test_v4.json"
input_path = "/content/drive/MyDrive/Final/Dataset/HotpotQA/hotpotqa_test_v2.json"
output_path = "/content/hotpot_QD_v6.json"
model = "gpt-4o-mini"  # Replace with your model if needed

# 접속사와 콤마를 기준으로 분리하는 함수
def split_sentence(text):
    # 예외 처리할 접속사 패턴
    exception_patterns = [
#        r'\b(?:both|either|neither|not only)\b.*?\b(?:and|or|nor)\b',
        r'\b(?:not only)\b.*?\b(?:but also)\b'
    ]
    # 예외 패턴에 해당하는 부분을 찾아서 미리 제외
    for pattern in exception_patterns:
        text = re.sub(pattern, lambda match: match.group(0).replace(" ", "_"), text)  # 예외 접속사 사이의 공백을 "_"로 치환

    # 일반적인 접속사와 콤마를 기준으로 분리
    conjunctions = r'\b(?:and|or|but|so|nor|yet)\b'
    parts = re.split(fr'\s*(?:{conjunctions}|,)\s*', text)

    # "_"를 원래 공백으로 복구
    parts = [part.replace("_", " ").strip() for part in parts if part.strip()]
    return parts

# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.
When you fill in a sentence (subquery) in Separate Parts, always find it in the words in the original question and fill it in.


Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "Who has won more awards, Dan Schneider or Helen Hunt?"
->
- subquery: Who has won more awards than Helen Hunt?
- subquery: Who has won more awards than Dan Schneider?

Example:
Original Query: "What do the Church of St. Mary of the Spring and Hagia Irene have in common?"
->
- subquery: What do the Church of St. Mary of the Spring have in common?
- subquery: What do Hagia Irene have in common?

One more example:
Original Query: "Which band, Matthew Good Band or Switchfoot is still together?"
->
- subquery: Which band, Matthew Good Band is still together?
- subquery: Which band, Switchfoot is still together?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

# LLM 호출 예시
def get_subqueries2(original_query, model="gpt-4o-mini"):
    split_results = split_sentence(original_query)
    prompt = generate_subqueries2(original_query, split_results)

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # 각 서브쿼리의 앞에 있는 "subquery:"와 "-"를 제거하고 리스트로 반환
    subqueries = [
        query.replace("- subquery: ", "").strip()  # "subquery:" 제거
        for query in response.choices[0].message.content.splitlines() if query.strip()
    ]
    return subqueries

# JSON 데이터 처리 함수
def process_json(data):
    processed_entries = []
    tmp = 0
    for entry in data:
        tmp += 1
        if tmp < 100:
            continue

        processed_entry = {
            "question": entry["question"],
            "ground_truth": entry["answer"],
            "ground_evidence": entry.get("supporting_facts_sentences", []),
            "decomposed_queries": get_subqueries2(entry["question"])
        }
        processed_entries.append(processed_entry)
    return processed_entries

# Load and process JSON
with open(input_path, 'r') as file:
    data = json.load(file)

processed_data = process_json(data)

# Save the updated JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print("JSON file updated and saved successfully.")


JSON file updated and saved successfully.


중복문장 제거

In [ ]:
import json

# JSON 파일 로드
with open('/content/hotpot_QD_v6.json', 'r') as file:
    data = json.load(file)

# 중복된 decomposed_queries 제거 함수 정의
def remove_exact_substrings(decomposed_queries):
    unique_queries = decomposed_queries[:]

    # 모든 쌍의 문장을 비교하여 짧은 문장의 마지막 한 글자 전까지가 긴 문장에 포함되는 경우 제거
    for i in range(len(decomposed_queries)):
        for j in range(i + 1, len(decomposed_queries)):
            short, long = (decomposed_queries[i], decomposed_queries[j]) if len(decomposed_queries[i]) < len(decomposed_queries[j]) else (decomposed_queries[j], decomposed_queries[i])
            # 짧은 문장의 마지막 한 글자 전까지가 긴 문장에 포함되는지 확인
            if short[:-1] in long and short in unique_queries:
                unique_queries.remove(short)

    return unique_queries

# 각 데이터 항목에 대해 중복 제거 적용
for item in data:
    item['decomposed_queries'] = remove_exact_substrings(item['decomposed_queries'])

# 수정된 데이터를 새 JSON 파일로 저장
with open('/content/hotpot_QD_v7.json', 'w') as file:
    json.dump(data, file, indent=4)

print("중복이 제거된 decomposed_queries가 저장된 파일이 생성되었습니다.")


중복이 제거된 decomposed_queries가 저장된 파일이 생성되었습니다.


In [ ]:
from langchain_openai import OpenAIEmbeddings

# FAISS 인덱스 파일 로드 (DB)
embeddings = OpenAIEmbeddings()
file_path = '/content/drive/MyDrive/Jimin/MultihopRAG_vectorstore_250_v4.index'

loaded_vectorstore = FAISS.load_local(
    folder_path=file_path,
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

Retriever 설정 및 Top-k 설정 (하단)

In [ ]:
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI


# Define LLM
answer_llm = ChatOpenAI(model_name="gpt-4o", temperature=0.1) # or gpt-4o-mini gpt-3.5-turbo


# Define prompt template
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
Please respond based only on the provided context, without using any prior knowledge.
Don't answer in a sentence form. Just answer with the keyword of the answer and only just one word.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": RunnablePassthrough(),  "question": RunnablePassthrough()}
    | prompt
    | answer_llm
    | StrOutputParser()
)